In [6]:
import numpy as np
from setup_mnist import *


In [21]:
data = MNIST()
X_exp = np.load("multiclass_data_2/X_exp.npy")
Y_exp = np.load("multiclass_data_2/Y_exp.npy")

In [22]:
X_exp.shape

(100, 784)

In [23]:
X_exp = X_exp.reshape(-1, 28, 28, 1)
Y_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Y_exp])

In [27]:
min(X_exp.reshape(-1,))

-0.5

In [30]:
a = np.ones(10)

In [31]:
a = np.expand_dims(a, axis=0)
a.shape

(1, 10)

In [9]:
model_dir = "deep_networks"
models = [conv_net(1, model_dir + "/conv1"), conv_net(0, model_dir + "/conv2"), 
          multilayer(4, 128, model_dir + "/mlp1"), multilayer(2, 256, model_dir + "/mlp2"),
          multilayer(0, 0, model_dir + "/zero_layer")]

In [24]:
[model.evaluate(X_exp, Y_exp) for model in models]

100/100 [==============================] - 0s 59us/step


[[0.0014530007843859493, 1.0],
 [0.015109961223934078, 0.99],
 [0.0056337832822464405, 1.0],
 [0.007803243593079969, 1.0],
 [0.06261769026517867, 1.0]]